# 🎯 Enhanced UFC Betting Analysis System

**Professional UFC betting with probability calibration and optimized models**

## 📋 Workflow:
1. **Cell 1**: Setup & Load Optimized Models
2. **Cell 2**: Auto-Fetch Live Odds from Sportsbooks
3. **Cell 3**: Generate Calibrated Predictions
4. **Cell 4**: Smart Bankroll Management
5. **Cell 5**: Save Bets & Track Performance

---

In [13]:
# 🚀 CELL 1: ENHANCED SETUP WITH PROVEN BETTING RULES
# =================================================
# Load optimized 32-feature model with NO CALIBRATION
# Using conservative betting rules based on historical performance

import pandas as pd
import numpy as np
import joblib
import json
import warnings
import sys
import os
import uuid
import shutil
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
from sklearn.calibration import CalibratedClassifierCV
from sklearn.isotonic import IsotonicRegression
warnings.filterwarnings('ignore')

# Add project paths
sys.path.append('../..')
sys.path.append('../../src')

print("🎯 UFC BETTING SYSTEM - CONSERVATIVE RULES")
print("=" * 50)

# ==================== CONFIGURATION ====================
# Your API key for The Odds API
YOUR_ODDS_API_KEY = "5100c18e74058e57c1d33a747e8c2be1"

# Your current bankroll (update this!)
CURRENT_BANKROLL = float(input("💰 Enter your current bankroll: $"))

# Probability calibration settings
USE_CALIBRATION = False  # NO CALIBRATION - use raw model predictions
CALIBRATION_STRENGTH = 1.0  # NO CALIBRATION - 100% of model confidence preserved

# Risk management settings (PROVEN THRESHOLDS)
MIN_EV_FOR_BET = 0.01     # 1% minimum EV
MAX_EV_FOR_BET = 0.15     # 15% maximum EV (avoid "too good to be true")
MIN_ODDS = 1.40           # Avoid heavy favorites
MAX_ODDS = 5.00           # Avoid extreme longshots
MAX_TOTAL_EXPOSURE = 0.12 # Max 12% of bankroll at risk

print(f"\n💳 Bankroll: ${CURRENT_BANKROLL:,.2f}")
print(f"📊 Max exposure: ${CURRENT_BANKROLL * MAX_TOTAL_EXPOSURE:,.2f}")
print(f"🎯 EV range: {MIN_EV_FOR_BET:.0%} to {MAX_EV_FOR_BET:.0%}")
print(f"📈 Odds range: {MIN_ODDS:.2f} to {MAX_ODDS:.2f}")

print("\n📋 BETTING RULES:")
print("   Singles: EV ∈ [1%, 15%], Odds ∈ [1.40, 5.00]")
print("   Stake: min(max(0.25×Kelly, 0.25% BR), 1% BR)")
print("   Parlays: Only if <2 singles, 3% min EV per leg")
print("   Parlay stake: 0.1-0.25% of bankroll")

def load_optimized_models():
    """Load the optimized 32-feature model if available."""
    
    # Check for optimized model first
    optimized_path = Path('../../model/optimized/ufc_model_optimized_latest.joblib')
    selector_path = Path('../../model/optimized/feature_selector_latest.json')
    
    if optimized_path.exists() and selector_path.exists():
        print("\n🚀 Loading OPTIMIZED model (32 features)...")
        model = joblib.load(optimized_path)
        
        # Load feature selector
        from ufc_predictor.models.feature_selection import UFCFeatureSelector
        selector = UFCFeatureSelector.load(str(selector_path))
        
        print(f"✅ Optimized model loaded")
        print(f"   • Features: {len(selector.selected_features)}")
        print(f"   • Proven thresholds active")
        return model, selector, 'optimized'
    else:
        print("\n📊 Loading standard model...")
        # Fallback to standard model loading
        return load_standard_models()

def load_standard_models():
    """Load standard models with all features."""
    model_dir = Path('../..')
    training_dirs = [d for d in model_dir.iterdir() if d.is_dir() and d.name.startswith('training_')]
    
    if training_dirs:
        latest_dir = sorted(training_dirs, key=lambda x: x.name)[-1]
        version = latest_dir.name.replace('training_', '')
        
        winner_model = joblib.load(latest_dir / f'ufc_winner_model_tuned_{version}.joblib')
        print(f"✅ Standard model loaded (version: {version})")
        return winner_model, None, 'standard'
    
    # Final fallback
    model_files = list(Path('../../model').glob('rf_tuned_model*.pkl'))
    if model_files:
        model = joblib.load(model_files[0])
        print(f"✅ Fallback model loaded")
        return model, None, 'fallback'
    
    raise FileNotFoundError("No models found!")

def initialize_enhanced_system():
    """Initialize the enhanced betting system."""
    try:
        # Load models
        model, selector, model_type = load_optimized_models()
        
        # Load fighter data
        fighters_path = Path('../../model/ufc_fighters_engineered_corrected.csv')
        if not fighters_path.exists():
            fighters_path = Path('../../data/ufc_fighters_engineered_corrected.csv')
        
        if fighters_path.exists():
            fighters_df = pd.read_csv(fighters_path)
            print(f"✅ Loaded {len(fighters_df):,} fighter records")
        else:
            print("⚠️  No fighter data found - predictions will be limited")
            fighters_df = None
        
        # Initialize API client
        print(f"\n🔌 Initializing Odds API client...")
        from ufc_predictor.betting.odds_api_integration import UFCOddsAPIClient
        odds_client = UFCOddsAPIClient(YOUR_ODDS_API_KEY)
        print(f"✅ API client ready")
        
        return {
            'model': model,
            'selector': selector,
            'model_type': model_type,
            'fighters_df': fighters_df,
            'odds_client': odds_client,
            'bankroll': CURRENT_BANKROLL,
            'calibration': USE_CALIBRATION,
            'calibration_strength': CALIBRATION_STRENGTH
        }
        
    except Exception as e:
        print(f"❌ Initialization failed: {e}")
        return None

# Initialize the system
print("\n🏗️ Initializing betting system with proven rules...")
betting_system = initialize_enhanced_system()

if betting_system:
    print("\n✅ SYSTEM READY!")
    print(f"   • Model: {betting_system['model_type'].upper()}")
    print(f"   • Calibration: DISABLED (raw predictions)")
    print(f"   • Bankroll: ${betting_system['bankroll']:,.2f}")
    print(f"   • Strategy: Conservative (proven thresholds)")
    print(f"\n🎮 Ready for betting analysis!")
else:
    print("\n❌ SYSTEM FAILED TO INITIALIZE")

🎯 UFC BETTING SYSTEM - CONSERVATIVE RULES

💳 Bankroll: $17.00
📊 Max exposure: $2.04
🎯 EV range: 1% to 15%
📈 Odds range: 1.40 to 5.00

📋 BETTING RULES:
   Singles: EV ∈ [1%, 15%], Odds ∈ [1.40, 5.00]
   Stake: min(max(0.25×Kelly, 0.25% BR), 1% BR)
   Parlays: Only if <2 singles, 3% min EV per leg
   Parlay stake: 0.1-0.25% of bankroll

🏗️ Initializing betting system with proven rules...

🚀 Loading OPTIMIZED model (32 features)...
✅ Optimized model loaded
   • Features: 32
   • Proven thresholds active
✅ Loaded 4,378 fighter records

🔌 Initializing Odds API client...
✅ API client ready

✅ SYSTEM READY!
   • Model: OPTIMIZED
   • Calibration: DISABLED (raw predictions)
   • Bankroll: $17.00
   • Strategy: Conservative (proven thresholds)

🎮 Ready for betting analysis!


In [14]:
# 📊 CELL 2: FETCH UFC 319 ODDS FROM THE ODDS API
# ====================================================
# Gets UFC 319 fight odds - ONLY the exact matchups for August 16, 2025

import sys
import warnings
from datetime import datetime, timedelta, timezone
import requests
warnings.filterwarnings('ignore')

sys.path.append('../..')
from src.ufc_predictor.betting.odds_api_integration import UFCOddsAPIClient

def get_ufc_319_odds():
    """Get UFC 319 odds from The Odds API."""
    print("🔍 FETCHING UFC 319 ODDS FROM THE ODDS API")
    print("=" * 50)
    
    if not betting_system:
        raise Exception("❌ System not initialized. Run Cell 1 first.")
    
    # Initialize API client
    api_client = betting_system['odds_client']
    
    print(f"📅 Today: {datetime.now().strftime('%A, %B %d, %Y')}")
    print(f"🌐 Connecting to The Odds API...")
    print(f"🔍 Looking for UFC 319 fights (August 16, 2025)...\n")
    
    try:
        # Get all MMA events from the API
        all_events = api_client.get_ufc_odds(region='au')
        
        print(f"📊 Total MMA fights in API: {len(all_events)}")
        
        # EXACT UFC 319 fight card matchups (August 16, 2025)
        known_ufc_319_fights = {
            # Main Card - EXACT matchups
            ("Dricus du Plessis", "Khamzat Chimaev"): "Main Event - Title Fight",
            ("Lerone Murphy", "Aaron Pico"): "Co-Main Event",
            ("Geoff Neal", "Carlos Prates"): "Main Card",
            ("Geoffrey Neal", "Carlos Prates"): "Main Card",  # Alternative spelling
            ("Jared Cannonier", "Michael Page"): "Main Card",
            ("Tim Elliott", "Kai Asakura"): "Main Card",
            # Preliminary Card
            ("Bobby Green", "Diego Ferreira"): "Prelims",
            ("King Green", "Carlos Diego Ferreira"): "Prelims",  # Alternative names
            ("Gerald Meerschaert", "Michal Oleksiejczuk"): "Prelims",
            ("Jessica Andrade", "Lupita Godinez"): "Prelims",
            ("Alexander Hernandez", "Chase Hooper"): "Prelims",
            ("Alex Hernandez", "Chase Hooper"): "Prelims",  # Alternative spelling
            ("Edson Barboza", "Drakkar Klose"): "Prelims",
            ("Nursulton Ruziboev", "Bryan Battle"): "Prelims",
            ("Nursultan Ruziboev", "Bryan Battle"): "Prelims",  # Alternative spelling
            ("Karine Silva", "Dione Barbosa"): "Prelims",
            ("Alibi Idiris", "Joseph Morales"): "Early Prelims",
        }
        
        # Look for UFC 319 fights - August 16-17, 2025 (accounting for timezone differences)
        ufc_319_start = datetime(2025, 8, 16, 0, 0, 0, tzinfo=timezone.utc)
        ufc_319_end = datetime(2025, 8, 18, 0, 0, 0, tzinfo=timezone.utc)
        
        ufc_319_fights_found = []
        all_dates_found = set()
        
        print(f"🔍 Searching for EXACT UFC 319 matchups...\n")
        
        for fight in all_events:
            fighter_a = fight.get('home_team', '')
            fighter_b = fight.get('away_team', '')
            
            if not fighter_a or not fighter_b:
                continue
            
            # Get event date
            commence_time = fight.get('commence_time', '')
            if commence_time:
                event_date = datetime.fromisoformat(commence_time.replace('Z', '+00:00'))
                all_dates_found.add(event_date.date())
                
                # Check if this is an EXACT UFC 319 matchup
                # Check both fighter order possibilities
                matchup1 = (fighter_a, fighter_b)
                matchup2 = (fighter_b, fighter_a)
                
                # Also check with lowercase and partial matches for flexibility
                is_ufc_319_fight = False
                card_position = None
                
                for known_matchup, position in known_ufc_319_fights.items():
                    known_a, known_b = known_matchup
                    
                    # Check exact match (both orders)
                    if ((fighter_a.lower() == known_a.lower() and fighter_b.lower() == known_b.lower()) or
                        (fighter_a.lower() == known_b.lower() and fighter_b.lower() == known_a.lower())):
                        is_ufc_319_fight = True
                        card_position = position
                        break
                    
                    # Check if last names match (for name variations)
                    fighter_a_last = fighter_a.split()[-1].lower() if fighter_a.split() else ""
                    fighter_b_last = fighter_b.split()[-1].lower() if fighter_b.split() else ""
                    known_a_last = known_a.split()[-1].lower() if known_a.split() else ""
                    known_b_last = known_b.split()[-1].lower() if known_b.split() else ""
                    
                    if ((fighter_a_last == known_a_last and fighter_b_last == known_b_last) or
                        (fighter_a_last == known_b_last and fighter_b_last == known_a_last)):
                        is_ufc_319_fight = True
                        card_position = position
                        break
                
                # Only add if it's a confirmed UFC 319 fight
                if is_ufc_319_fight:
                    # Check if not already added
                    already_added = any(
                        f['fighter_a'] == fighter_a and f['fighter_b'] == fighter_b
                        for f in ufc_319_fights_found
                    )
                    
                    if not already_added:
                        ufc_319_fights_found.append({
                            'fighter_a': fighter_a,
                            'fighter_b': fighter_b,
                            'date': event_date,
                            'card_position': card_position,
                            'raw_data': fight
                        })
                        date_str = event_date.strftime('%Y-%m-%d %H:%M UTC')
                        print(f"  ✅ Found: {fighter_a} vs {fighter_b} ({card_position}) - {date_str}")
        
        # Count unique matchups (remove alternative spellings)
        unique_matchups = set()
        for known_matchup in known_ufc_319_fights.keys():
            # Normalize to last names for counting
            a, b = known_matchup
            a_last = a.split()[-1].lower()
            b_last = b.split()[-1].lower()
            unique_matchups.add(tuple(sorted([a_last, b_last])))
        
        expected_fights = len(unique_matchups)
        
        if not ufc_319_fights_found:
            print(f"\n⚠️ No UFC 319 fights found")
            print(f"All dates in API: {sorted(all_dates_found)[:10]}...")  # Show first 10 dates
            return {}, None
        
        print(f"\n📊 UFC 319 FIGHTS FOUND: {len(ufc_319_fights_found)}/{expected_fights}")
        print("=" * 50)
        
        # Extract odds for each fight
        odds_data = {}
        
        # Sort by card position
        card_order = ["Main Event - Title Fight", "Co-Main Event", "Main Card", "Prelims", "Early Prelims"]
        ufc_319_fights_found.sort(key=lambda x: card_order.index(x['card_position']) if x['card_position'] in card_order else 99)
        
        for i, fight_info in enumerate(ufc_319_fights_found, 1):
            fighter_a = fight_info['fighter_a']
            fighter_b = fight_info['fighter_b']
            card_pos = fight_info['card_position']
            raw_fight = fight_info['raw_data']
            
            # Get best odds from bookmakers
            best_odds_a = None
            best_odds_b = None
            
            for bookmaker in raw_fight.get('bookmakers', []):
                for market in bookmaker.get('markets', []):
                    if market.get('key') == 'h2h':
                        for outcome in market.get('outcomes', []):
                            if outcome['name'] == fighter_a:
                                price = outcome.get('price', 0)
                                if price and (not best_odds_a or price > best_odds_a):
                                    best_odds_a = price
                            elif outcome['name'] == fighter_b:
                                price = outcome.get('price', 0)
                                if price and (not best_odds_b or price > best_odds_b):
                                    best_odds_b = price
            
            # Use first bookmaker if no best odds
            if not best_odds_a or not best_odds_b:
                bookmakers = raw_fight.get('bookmakers', [])
                if bookmakers:
                    for market in bookmakers[0].get('markets', []):
                        if market.get('key') == 'h2h':
                            outcomes = market.get('outcomes', [])
                            if len(outcomes) >= 2:
                                if not best_odds_a:
                                    best_odds_a = outcomes[0].get('price', 2.0)
                                if not best_odds_b:
                                    best_odds_b = outcomes[1].get('price', 2.0)
                                break
            
            # Default odds if still not found
            if not best_odds_a:
                best_odds_a = 2.0
            if not best_odds_b:
                best_odds_b = 2.0
            
            # Store odds with CORRECT KEY FORMAT (spaces, not underscores!)
            fight_key = f"{fighter_a} vs {fighter_b}"  # FIX: Use spaces, not underscores!
            odds_data[fight_key] = {
                'fighter_a': fighter_a,
                'fighter_b': fighter_b,
                'fighter_a_decimal_odds': best_odds_a,
                'fighter_b_decimal_odds': best_odds_b
            }
            
            marker = "⭐" if best_odds_a < best_odds_b else "🎲"
            print(f"{i:2}. [{card_pos:15}] {fighter_a} ({best_odds_a:.2f}) vs {fighter_b} ({best_odds_b:.2f}) {marker}")
        
        # Show missing fights
        print(f"\n⚠️ UFC 319 FIGHTS NOT FOUND IN API:")
        found_matchups = set()
        for fight in ufc_319_fights_found:
            a_last = fight['fighter_a'].split()[-1].lower()
            b_last = fight['fighter_b'].split()[-1].lower()
            found_matchups.add(tuple(sorted([a_last, b_last])))
        
        for matchup in unique_matchups:
            if matchup not in found_matchups:
                # Find original names for display
                for known_matchup, position in known_ufc_319_fights.items():
                    a, b = known_matchup
                    a_last = a.split()[-1].lower()
                    b_last = b.split()[-1].lower()
                    if tuple(sorted([a_last, b_last])) == matchup:
                        print(f"  ❌ {a} vs {b} ({position})")
                        break
        
        print(f"\n📊 SUMMARY:")
        print(f"   • Event: UFC 319")
        print(f"   • Date: August 16, 2025")
        print(f"   • Location: United Center, Chicago")
        print(f"   • Expected fights: {expected_fights}")
        print(f"   • Fights found in API: {len(odds_data)}")
        print(f"   • Coverage: {len(odds_data)/expected_fights*100:.1f}%")
        print(f"   • Source: The Odds API")
        
        # DEBUG: Show the actual keys being created
        print(f"\n🔍 DEBUG - Odds data keys created:")
        for key in list(odds_data.keys())[:3]:
            print(f"   • '{key}'")
        
        return odds_data, {
            'name': 'UFC 319',
            'date': '2025-08-16',
            'fights': [f"{fight['fighter_a']} vs {fight['fighter_b']}" for fight in ufc_319_fights_found],
            'total_fight_count': expected_fights,
            'fights_with_odds': len(odds_data),
            'source': 'The Odds API'
        }
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        print(f"Debug: {traceback.format_exc()}")
        return {}, None

# ================== MAIN EXECUTION ==================
print("🎯 UFC 319 SPECIFIC ODDS FETCHER")
print("=" * 50)
print("Looking for EXACT UFC 319 matchups only\n")

# Get UFC 319 odds
odds_data, event_info = get_ufc_319_odds()

if odds_data and event_info:
    # Set global variables for other cells
    EVENT_NAME = event_info['name']
    TARGET_FIGHTS = event_info['fights']
    
    print(f"\n✅ UFC 319 ODDS LOADED!")
    print(f"   • Event: {EVENT_NAME}")
    print(f"   • Date: {event_info['date']}")
    print(f"   • Fights found: {len(odds_data)}")
    print(f"   • Expected: {event_info['total_fight_count']}")
    print(f"\n🎮 Ready for prediction analysis!")
else:
    print(f"\n❌ Unable to load UFC 319 odds")
    odds_data = {}
    EVENT_NAME = "UFC 319"
    TARGET_FIGHTS = []

🎯 UFC 319 SPECIFIC ODDS FETCHER
Looking for EXACT UFC 319 matchups only

🔍 FETCHING UFC 319 ODDS FROM THE ODDS API
📅 Today: Wednesday, August 13, 2025
🌐 Connecting to The Odds API...
🔍 Looking for UFC 319 fights (August 16, 2025)...

🔄 Fetching UFC odds from The Odds API...
   Region: au
   URL: https://api.the-odds-api.com/v4/sports/mma_mixed_martial_arts/odds
📊 API Usage - Remaining: 306, Used: 194
✅ Found 86 UFC events
📊 Total MMA fights in API: 86
🔍 Searching for EXACT UFC 319 matchups...

  ✅ Found: Alex Hernandez vs Chase Hooper (Prelims) - 2025-08-16 22:00 UTC
  ✅ Found: Nursultan Ruziboev vs Bryan Battle (Prelims) - 2025-08-16 22:00 UTC
  ✅ Found: King Green vs Carlos Diego Ferreira (Prelims) - 2025-08-16 22:00 UTC
  ✅ Found: Edson Barboza vs Drakkar Klose (Prelims) - 2025-08-16 22:00 UTC
  ✅ Found: Gerald Meerschaert vs Michal Oleksiejczuk (Prelims) - 2025-08-16 22:00 UTC
  ✅ Found: Jessica Andrade vs Lupita Godinez (Prelims) - 2025-08-16 22:00 UTC
  ✅ Found: Karine Silva vs D

In [15]:
# 🧠 CELL 3: MULTI-BET ANALYSIS WITH CONDITIONAL PARLAYS (FIXED V3)
# ==================================================================
# Generate predictions with sophisticated multi-bet strategy
# FIXED: Module reload to handle updated configurations without kernel restart

import sys
import pandas as pd
import numpy as np
import json
import importlib
sys.path.append('../../src')

# Force reload modules to get updated configurations (fixes KeyError: 'odds_min')
print("🔄 Reloading modules to get latest configurations...")
if 'ufc_predictor.betting.selection' in sys.modules:
    importlib.reload(sys.modules['ufc_predictor.betting.selection'])
if 'ufc_predictor.betting.multi_bet_orchestrator' in sys.modules:
    importlib.reload(sys.modules['ufc_predictor.betting.multi_bet_orchestrator'])
if 'ufc_predictor.betting.enhanced_correlation' in sys.modules:
    importlib.reload(sys.modules['ufc_predictor.betting.enhanced_correlation'])
if 'ufc_predictor.betting.parlay_builder' in sys.modules:
    importlib.reload(sys.modules['ufc_predictor.betting.parlay_builder'])
print("✅ Modules reloaded with latest configurations")

# IMPORTANT: Import AFTER reload to get the updated modules
from ufc_predictor.betting.multi_bet_orchestrator import MultiBetOrchestrator

def find_fighter_fuzzy(fighter_name, fighters_df):
    """Find fighter with fuzzy matching for better name resolution."""
    import difflib
    
    # Normalize the search name
    search_name = fighter_name.strip().lower()
    
    # Try exact match first
    fighters_df['Name_normalized'] = fighters_df['Name'].str.strip().str.lower()
    exact_match = fighters_df[fighters_df['Name_normalized'] == search_name]
    if not exact_match.empty:
        return exact_match.iloc[0]
    
    # Try fuzzy matching
    all_names = fighters_df['Name_normalized'].tolist()
    close_matches = difflib.get_close_matches(search_name, all_names, n=1, cutoff=0.8)
    
    if close_matches:
        matched_name = close_matches[0]
        matched_fighter = fighters_df[fighters_df['Name_normalized'] == matched_name]
        if not matched_fighter.empty:
            print(f"      📝 Fuzzy matched '{fighter_name}' to '{matched_fighter.iloc[0]['Name']}'")
            return matched_fighter.iloc[0]
    
    # Try partial match (last name)
    last_name = search_name.split()[-1] if search_name.split() else search_name
    partial_matches = fighters_df[fighters_df['Name_normalized'].str.contains(last_name, na=False)]
    if len(partial_matches) == 1:
        print(f"      📝 Partial matched '{fighter_name}' to '{partial_matches.iloc[0]['Name']}'")
        return partial_matches.iloc[0]
    
    return None

def predict_fight_winner_robust(fighter1_name, fighter2_name, fighters_df, model, feature_columns):
    """
    Robust fight prediction with better error handling and fighter matching.
    """
    
    def get_fighter_stats(fighter_name: str):
        fighter_row = find_fighter_fuzzy(fighter_name, fighters_df)
        if fighter_row is None:
            raise ValueError(f"Fighter '{fighter_name}' not found")
        return fighter_row
    
    def create_differential_features(blue_stats: pd.Series, red_stats: pd.Series) -> dict:
        diff_features = {}
        
        for blue_col in blue_stats.index:
            if blue_col.startswith('blue_') and 'url' not in blue_col and 'Name' not in blue_col:
                red_col = blue_col.replace('blue_', 'red_')
                base_name = blue_col.replace('blue_', '')
                
                if red_col in red_stats.index:
                    diff_col_name = base_name.lower().replace(' ', '_').replace('.', '') + '_diff'
                    try:
                        diff_features[diff_col_name] = float(blue_stats[blue_col]) - float(red_stats[red_col])
                    except:
                        diff_features[diff_col_name] = 0
        
        return diff_features
    
    def get_prediction_from_perspective(f1, f2):
        """Get prediction with f1 as blue corner, f2 as red corner."""
        fighter1_stats = get_fighter_stats(f1)
        fighter2_stats = get_fighter_stats(f2)
        
        # Create blue and red corner statistics with proper prefixes
        blue_stats = fighter1_stats.add_prefix('blue_')
        red_stats = fighter2_stats.add_prefix('red_')
        
        # Create differential features
        diff_features = create_differential_features(blue_stats, red_stats)
        
        # Combine all features
        all_features = {}
        
        # Add blue features
        for col in blue_stats.index:
            if 'url' not in col.lower() and 'name' not in col.lower():
                try:
                    all_features[col] = float(blue_stats[col])
                except:
                    all_features[col] = 0
        
        # Add red features  
        for col in red_stats.index:
            if 'url' not in col.lower() and 'name' not in col.lower():
                try:
                    all_features[col] = float(red_stats[col])
                except:
                    all_features[col] = 0
        
        # Add diff features
        all_features.update(diff_features)
        
        # Add Round feature if needed
        if 'Round' in feature_columns and 'Round' not in all_features:
            all_features['Round'] = 3  # Default to 3 rounds
        
        # Create prediction DataFrame with correct column order
        prediction_df = pd.DataFrame([all_features]).reindex(columns=feature_columns, fill_value=0)
        
        # Get prediction
        if hasattr(model, 'predict_proba'):
            probs = model.predict_proba(prediction_df)[0]
            return probs[1] if len(probs) == 2 else probs[0]
        else:
            return model.predict(prediction_df)[0]
    
    try:
        # Prediction 1: fighter1 is blue corner, fighter2 is red corner
        prob_f1_wins_as_blue = get_prediction_from_perspective(fighter1_name, fighter2_name)
        
        # Prediction 2: fighter2 is blue corner, fighter1 is red corner  
        prob_f2_wins_as_blue = get_prediction_from_perspective(fighter2_name, fighter1_name)
        
        # From the second perspective, probability of fighter1 winning is 1 - fighter2's win probability
        prob_f1_wins_as_red = 1 - prob_f2_wins_as_blue
        
        # Average the probabilities for symmetrical prediction
        final_prob_f1_wins = (prob_f1_wins_as_blue + prob_f1_wins_as_red) / 2
        final_prob_f2_wins = 1 - final_prob_f1_wins
        
        return {
            'fighter_1': fighter1_name,
            'fighter_2': fighter2_name,
            'prob_1': final_prob_f1_wins,
            'prob_2': final_prob_f2_wins,
            'winner': fighter1_name if final_prob_f1_wins > final_prob_f2_wins else fighter2_name,
            'confidence': max(final_prob_f1_wins, final_prob_f2_wins)
        }
        
    except Exception as e:
        return {'error': str(e)}

def generate_multi_bet_predictions_fixed():
    """Generate predictions using the sophisticated multi-bet system with robust fighter matching."""
    print("🧠 MULTI-BET PREDICTION ENGINE V3.3 (MODULE RELOAD FIXED)")
    print("=" * 50)
    print("✅ Module reload: Fixed KeyError without kernel restart")
    print("✅ Conservative rules: EV [1%, 15%], Odds [1.40, 5.00]")
    print("📊 Strategy: Conditional Parlay with Correlation Analysis")
    print("🎯 Singles: 1-15% EV range with odds filtering")
    print("🎰 Parlays: Activated when <2 singles qualify")
    print("-" * 50)
    
    if not betting_system:
        raise Exception("❌ System not initialized")
    
    if not odds_data:
        raise Exception("❌ No odds data available")
    
    # Prepare predictions DataFrame
    predictions_list = []
    successful_predictions = 0
    
    for fight_key, fight_odds in odds_data.items():
        fighter_a = fight_odds['fighter_a']
        fighter_b = fight_odds['fighter_b']
        
        print(f"\n🥊 {fight_key}")
        print(f"   Analyzing: {fighter_a} vs {fighter_b}")
        
        try:
            # Generate prediction using ROBUST logic with fuzzy matching
            if betting_system['fighters_df'] is not None:
                
                # Use the model's feature columns (32 features for optimized)
                if betting_system['selector']:
                    feature_cols = betting_system['selector'].selected_features
                    print(f"   📊 Using {len(feature_cols)} model features")
                else:
                    # Load from JSON if available
                    try:
                        with open('../../model/winner_model_columns.json', 'r') as f:
                            feature_cols = json.load(f)
                    except:
                        print(f"   ❌ No feature columns found")
                        prob_a = 0.5
                        
                # Use robust prediction function with fuzzy matching
                pred_result = predict_fight_winner_robust(
                    fighter_a, fighter_b,
                    betting_system['fighters_df'],
                    betting_system['model'],
                    feature_cols
                )
                
                if 'error' not in pred_result:
                    prob_a = pred_result['prob_1']
                    print(f"   ✅ Model prediction successful: {prob_a:.3f}")
                    successful_predictions += 1
                else:
                    print(f"   ⚠️  Prediction error: {pred_result['error']}")
                    # Use market odds as fallback
                    market_prob_a = 1 / fight_odds['fighter_a_decimal_odds']
                    prob_a = market_prob_a
                    print(f"   📈 Using market odds: {prob_a:.3f}")
                        
            else:
                # Demo prediction with market edge
                market_prob_a = 1 / fight_odds['fighter_a_decimal_odds']
                prob_a = market_prob_a + np.random.randn() * 0.05
                prob_a = np.clip(prob_a, 0.2, 0.8)
                print(f"   🎲 Demo prediction: {prob_a:.3f}")
            
            prob_b = 1 - prob_a
            
            # Confidence based on probability distance from 50%
            confidence = min(0.85, 0.5 + abs(prob_a - 0.5))
            
            predictions_list.append({
                'fighter_a': fighter_a,
                'fighter_b': fighter_b,
                'prob_a': prob_a,
                'prob_b': prob_b,
                'confidence': confidence,
                'event': EVENT_NAME,
                'fight_id': fight_key.replace(' ', '_'),
                'weight_class': 'unknown',
                'card_position': len(predictions_list) + 1
            })
            
            print(f"   📊 Final probability: {prob_a:.1%} vs {prob_b:.1%}")
            print(f"   🎯 Confidence: {confidence:.1%}")
            
            # Show EV calculation preview
            market_prob_a = 1 / fight_odds['fighter_a_decimal_odds']
            market_prob_b = 1 / fight_odds['fighter_b_decimal_odds']
            ev_a = (prob_a * fight_odds['fighter_a_decimal_odds']) - 1
            ev_b = (prob_b * fight_odds['fighter_b_decimal_odds']) - 1
            
            if ev_a > 0.01:
                print(f"   💰 Potential bet on {fighter_a}: EV={ev_a:.1%}, Odds={fight_odds['fighter_a_decimal_odds']:.2f}")
            if ev_b > 0.01:
                print(f"   💰 Potential bet on {fighter_b}: EV={ev_b:.1%}, Odds={fight_odds['fighter_b_decimal_odds']:.2f}")
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
            # Continue with next fight instead of crashing
            continue
    
    print(f"\n✅ Successfully predicted {successful_predictions}/{len(odds_data)} fights")
    
    if not predictions_list:
        print("\n❌ No predictions generated")
        return None
    
    predictions_df = pd.DataFrame(predictions_list)
    
    # Initialize multi-bet orchestrator (will use reloaded module with correct config)
    print(f"\n🚀 INITIALIZING MULTI-BET ORCHESTRATOR")
    print("-" * 50)
    orchestrator = MultiBetOrchestrator()
    
    # Run sophisticated analysis
    result = orchestrator.analyze_betting_opportunities(
        predictions_df, odds_data, betting_system['bankroll']
    )
    
    # Display results
    print(f"\n📊 MULTI-BET ANALYSIS RESULTS")
    print("=" * 50)
    print(f"🎯 Strategy: {result.strategy_used.upper()}")
    print(f"📋 Reason: {result.activation_reason}")
    print(f"💰 Total Exposure: ${result.total_exposure:.2f} ({result.total_exposure/betting_system['bankroll']:.1%})")
    print(f"📈 Expected Return: ${result.expected_return:.2f}")
    print(f"⚠️  Portfolio Risk: {result.portfolio_risk:.2f}")
    
    # Singles analysis
    if result.singles:
        print(f"\n💰 QUALIFIED SINGLES ({len(result.singles)})")
        print("-" * 40)
        for single in result.singles:
            # Use the new stake formula
            stake = orchestrator._calculate_single_stake(single, betting_system['bankroll'])
            print(f"• {single.fighter} vs {single.opponent}")
            print(f"  Model: {single.model_prob:.1%} | Market: {single.market_prob:.1%}")
            print(f"  EV: {single.ev:.1%} | Gap: {single.market_gap:.1%}")
            print(f"  Odds: {single.odds:.2f} | Confidence: {single.confidence:.1%}")
            print(f"  Stake: ${stake:.2f} (min(max(0.25×Kelly, 0.25% BR), 1% BR))")
            print()
    else:
        print(f"\n📭 NO QUALIFIED SINGLES")
        print("   • No bets meet conservative criteria:")
        print("   • EV must be in [1%, 15%]")
        print("   • Odds must be in [1.40, 5.00]")
        print("   • Market gap ≥ 2%")
    
    # Parlays analysis
    if result.parlays:
        print(f"\n🎲 SELECTED PARLAYS ({len(result.parlays)})")
        print("-" * 40)
        for i, parlay in enumerate(result.parlays, 1):
            print(f"{i}. {' + '.join(parlay.fighters)}")
            print(f"   Combined Prob: {parlay.combined_prob:.1%}")
            print(f"   Combined Odds: {parlay.combined_odds:.2f}")
            print(f"   Combined EV: {parlay.combined_ev:.1%}")
            print(f"   Correlation: {parlay.correlation:.3f}")
            print(f"   Confidence: {parlay.confidence_score:.1%}")
            print(f"   Stake: ${parlay.stake_amount:.2f} (0.1-0.25% BR)")
            print(f"   Expected Return: ${parlay.expected_return:.2f}")
            print()
    
    # Strategy explanation
    print(f"\n🔍 STRATEGY EXPLANATION")
    print("-" * 40)
    if result.strategy_used == 'singles_only':
        print("✅ Primary strategy activated")
        print(f"   • {len(result.singles)} singles meet conservative criteria")
        print("   • EV range [1%, 15%] (based on your winning history)")
        print("   • Odds range [1.40, 5.00] (avoiding extremes)")
        print("   • No parlays needed")
    elif result.strategy_used == 'conditional_parlays':
        print("🎰 Conditional parlay strategy activated")
        print(f"   • Only {len(result.singles)} single(s) qualified (<2 threshold)")
        print("   • Parlays use relaxed filters (3% EV per leg)")
        print("   • Max 3 legs per parlay")
        print("   • Fixed staking: 0.1-0.25% of bankroll")
    else:
        print("📭 No bets recommended")
        print("   • Opportunities don't meet conservative criteria")
        print("   • This is good - discipline protects bankroll!")
        print("   • Your historical data shows >15% EV bets lose")
    
    # Risk management summary
    print(f"\n⚠️  RISK MANAGEMENT")
    print("-" * 40)
    max_exposure = betting_system['bankroll'] * 0.12
    print(f"• Portfolio Limit: ${max_exposure:.2f} (12% of bankroll)")
    print(f"• Current Exposure: ${result.total_exposure:.2f}")
    print(f"• Remaining Capacity: ${max_exposure - result.total_exposure:.2f}")
    
    if result.parlays:
        avg_corr = np.mean([p.correlation for p in result.parlays])
        print(f"• Average Correlation: {avg_corr:.3f}")
    
    # Save result globally for other cells
    global multi_bet_result
    multi_bet_result = result
    
    return result

# Generate multi-bet predictions with MODULE RELOAD FIX
print("🎯 Running conservative betting analysis with MODULE RELOAD FIX...")
print("   ✅ Modules reloaded to get latest configuration")
print("   ✅ Import happens AFTER reload (critical!)")
print("   ✅ Conservative rules: EV [1%, 15%], Odds [1.40, 5.00]")
print("   ✅ Based on your historical performance data\n")

multi_bet_result = generate_multi_bet_predictions_fixed()

if multi_bet_result:
    print(f"\n✅ MULTI-BET ANALYSIS COMPLETE!")
    if multi_bet_result.total_exposure > 0:
        print(f"   💰 Found profitable opportunities within conservative criteria")
    else:
        print(f"   📭 No bets meet conservative criteria (discipline is key!)")
else:
    print(f"\n❌ Analysis failed")

🔄 Reloading modules to get latest configurations...
✅ Modules reloaded with latest configurations
🎯 Running conservative betting analysis with MODULE RELOAD FIX...
   ✅ Modules reloaded to get latest configuration
   ✅ Import happens AFTER reload (critical!)
   ✅ Conservative rules: EV [1%, 15%], Odds [1.40, 5.00]
   ✅ Based on your historical performance data

🧠 MULTI-BET PREDICTION ENGINE V3.3 (MODULE RELOAD FIXED)
✅ Module reload: Fixed KeyError without kernel restart
✅ Conservative rules: EV [1%, 15%], Odds [1.40, 5.00]
📊 Strategy: Conditional Parlay with Correlation Analysis
🎯 Singles: 1-15% EV range with odds filtering
🎰 Parlays: Activated when <2 singles qualify
--------------------------------------------------

🥊 Dricus Du Plessis vs Khamzat Chimaev
   Analyzing: Dricus Du Plessis vs Khamzat Chimaev
   📊 Using 32 model features
   ✅ Model prediction successful: 0.471
   📊 Final probability: 47.1% vs 52.9%
   🎯 Confidence: 52.9%
   💰 Potential bet on Dricus Du Plessis: EV=45.9%

In [16]:
# 💰 CELL 4: PORTFOLIO OPTIMIZATION WITH MULTI-BET STRATEGY
# ==================================================
# Generate optimized portfolio from multi-bet analysis results

def generate_multi_bet_portfolio():
    """Generate betting portfolio from multi-bet orchestrator results."""
    print("💰 MULTI-BET PORTFOLIO GENERATION")
    print("=" * 50)
    
    if not multi_bet_result:
        raise Exception("❌ No multi-bet analysis available. Run Cell 3 first.")
    
    bankroll = betting_system['bankroll']
    
    print(f"💳 Bankroll: ${bankroll:,.2f}")
    print(f"🎯 Strategy: {multi_bet_result.strategy_used.upper()}")
    print(f"📊 Total Exposure: ${multi_bet_result.total_exposure:.2f}")
    
    recommendations = []
    
    # Process singles
    if multi_bet_result.singles:
        print(f"\n🎯 SINGLES PORTFOLIO ({len(multi_bet_result.singles)})")
        print("-" * 40)
        
        for single in multi_bet_result.singles:
            # Calculate conservative Kelly stake
            edge = single.ev
            kelly_full = edge / (single.odds - 1) if single.odds > 1 else 0
            kelly_quarter = kelly_full * 0.25  # Quarter Kelly
            
            # Apply confidence scaling
            confidence_factor = 0.5 + (single.confidence * 0.5)
            kelly_adjusted = kelly_quarter * confidence_factor
            
            # Calculate stake
            stake = bankroll * kelly_adjusted
            
            # Apply limits
            max_single = bankroll * 0.05  # Max 5% per single
            stake = min(stake, max_single)
            
            # Minimum threshold
            if stake < bankroll * 0.005:  # 0.5% minimum
                stake = 0
            
            if stake > 0:
                recommendation = {
                    'type': 'SINGLE',
                    'fighter': single.fighter,
                    'opponent': single.opponent,
                    'event': single.event,
                    'probability': single.model_prob,
                    'odds': single.odds,
                    'ev': single.ev,
                    'confidence': single.confidence,
                    'bet_size': round(stake, 2),
                    'potential_profit': round(stake * (single.odds - 1), 2),
                    'market_gap': single.market_gap,
                    'is_favorite': single.is_favorite
                }
                
                recommendations.append(recommendation)
                
                print(f"\n✅ {single.fighter}")
                print(f"   vs {single.opponent}")
                print(f"   Stake: ${stake:.2f}")
                print(f"   Odds: {single.odds:.2f}")
                print(f"   EV: {single.ev:+.1%}")
                print(f"   Potential: ${stake * (single.odds - 1):.2f}")
    
    # Process parlays
    if multi_bet_result.parlays:
        print(f"\n🎲 PARLAY PORTFOLIO ({len(multi_bet_result.parlays)})")
        print("-" * 40)
        
        for i, parlay in enumerate(multi_bet_result.parlays, 1):
            if parlay.stake_amount > 0:
                recommendation = {
                    'type': 'PARLAY',
                    'fighters': ' + '.join(parlay.fighters),
                    'legs': parlay.legs,
                    'event': multi_bet_result.metadata.get('event', EVENT_NAME),
                    'probability': parlay.combined_prob,
                    'odds': parlay.combined_odds,
                    'ev': parlay.combined_ev,
                    'confidence': parlay.confidence_score,
                    'bet_size': parlay.stake_amount,
                    'potential_profit': parlay.expected_return,
                    'correlation': parlay.correlation,
                    'n_legs': parlay.n_legs
                }
                
                recommendations.append(recommendation)
                
                print(f"\n✅ Parlay {i}: {' + '.join(parlay.fighters)}")
                print(f"   Stake: ${parlay.stake_amount:.2f}")
                print(f"   Odds: {parlay.combined_odds:.2f}")
                print(f"   EV: {parlay.combined_ev:+.1%}")
                print(f"   Correlation: {parlay.correlation:.3f}")
                print(f"   Potential: ${parlay.expected_return:.2f}")
    
    # Portfolio summary
    print(f"\n📊 PORTFOLIO SUMMARY")
    print("=" * 50)
    
    if recommendations:
        singles = [r for r in recommendations if r['type'] == 'SINGLE']
        parlays = [r for r in recommendations if r['type'] == 'PARLAY']
        
        total_stake = sum(r['bet_size'] for r in recommendations)
        total_potential = sum(r['potential_profit'] for r in recommendations)
        avg_ev = sum(r['ev'] * r['bet_size'] for r in recommendations) / total_stake if total_stake > 0 else 0
        
        print(f"Singles: {len(singles)}")
        print(f"Parlays: {len(parlays)}")
        print(f"Total stake: ${total_stake:.2f}")
        print(f"Bankroll usage: {total_stake/bankroll*100:.1f}%")
        print(f"Expected value: {avg_ev:+.1%}")
        print(f"Potential profit: ${total_potential:.2f}")
        print(f"ROI: {(total_potential/total_stake)*100:.1f}%" if total_stake > 0 else "")
        
        # Risk assessment
        print(f"\n🎲 RISK PROFILE:")
        if multi_bet_result.strategy_used == 'singles_only':
            print(f"   • PRIMARY: Singles strategy (lowest risk)")
            print(f"   • All bets in 5-15% EV range")
        elif multi_bet_result.strategy_used == 'conditional_parlays':
            print(f"   • CONDITIONAL: Parlay strategy (moderate risk)")
            print(f"   • Max correlation: {max(p.correlation for p in multi_bet_result.parlays):.3f}" if multi_bet_result.parlays else "")
            print(f"   • Conservative staking (0.5% cap)")
        else:
            print(f"   • NO BETS: Strict filters working")
        
        print(f"   • Conservative Kelly (25% fraction)")
        print(f"   • Portfolio limit: 12% of bankroll")
        
    else:
        print("📭 No betting opportunities found")
        print("✅ This is good - strict filters protect bankroll!")
    
    return recommendations

# Generate portfolio
print("🎯 Generating optimized portfolio from multi-bet analysis...")
betting_portfolio = generate_multi_bet_portfolio()

if betting_portfolio:
    print(f"\n✅ PORTFOLIO READY!")
    print(f"   • {len(betting_portfolio)} total recommendations")
    print(f"   • Strategy: {multi_bet_result.strategy_used}")
else:
    print(f"\n📭 No bets recommended - discipline is key!")

🎯 Generating optimized portfolio from multi-bet analysis...
💰 MULTI-BET PORTFOLIO GENERATION
💳 Bankroll: $17.00
🎯 Strategy: CONDITIONAL_PARLAYS
📊 Total Exposure: $0.21

🎯 SINGLES PORTFOLIO (1)
----------------------------------------

✅ Michael Page
   vs Jared Cannonier
   Stake: $0.49
   Odds: 1.55
   EV: +7.5%
   Potential: $0.27

🎲 PARLAY PORTFOLIO (1)
----------------------------------------

✅ Parlay 1: Tim Elliott + Alex Hernandez + Gerald Meerschaert
   Stake: $0.04
   Odds: 38.42
   EV: +476.4%
   Correlation: 0.172
   Potential: $0.19

📊 PORTFOLIO SUMMARY
Singles: 1
Parlays: 1
Total stake: $0.53
Bankroll usage: 3.1%
Expected value: +42.9%
Potential profit: $0.46
ROI: 86.9%

🎲 RISK PROFILE:
   • CONDITIONAL: Parlay strategy (moderate risk)
   • Max correlation: 0.172
   • Conservative staking (0.5% cap)
   • Conservative Kelly (25% fraction)
   • Portfolio limit: 12% of bankroll

✅ PORTFOLIO READY!
   • 2 total recommendations
   • Strategy: conditional_parlays


In [18]:
# 📝 CELL 5: SAVE & TRACK WITH ADVANCED BET TRACKING SYSTEM
# =======================================================
# Professional bet tracking with deduplication and performance analysis

import sys
import os
from datetime import datetime
sys.path.append('../..')

# Import the comprehensive bet tracking system
from ufc_predictor.betting.bet_tracking import BetTracker

def save_betting_portfolio_advanced():
    """Save betting portfolio using the advanced tracking system."""
    print("📝 SAVING BETTING PORTFOLIO (ADVANCED)")
    print("=" * 50)
    
    if not betting_portfolio:
        print("📭 No bets to save")
        return []
    
    # Initialize the bet tracker
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    saved_bets = []
    event_date = datetime.now().strftime('%Y-%m-%d')
    
    print(f"📋 Processing {len(betting_portfolio)} recommendations...")
    print("-" * 40)
    
    for i, bet in enumerate(betting_portfolio, 1):
        # Prepare method prediction
        method_pred = "Decision"  # Default, could be enhanced with actual method predictions
        
        # Determine risk level based on EV
        if bet['ev'] > 0.10:
            risk_level = 'HIGH'
        elif bet['ev'] > 0.05:
            risk_level = 'MEDIUM'
        else:
            risk_level = 'LOW'
        
        # Handle both singles and parlays
        if bet['type'] == 'SINGLE':
            # Log single bet
            bet_id = tracker.log_bet_recommendation(
                event_name=EVENT_NAME,
                event_date=event_date,
                fighter=bet['fighter'],
                opponent=bet['opponent'],
                bet_type='SINGLE',
                decimal_odds=bet['odds'],
                american_odds=int((bet['odds'] - 1) * 100) if bet['odds'] >= 2 else int(-100/(bet['odds']-1)),
                bet_size=bet['bet_size'],
                expected_value=bet['ev'],
                expected_return=bet['potential_profit'],
                model_probability=bet['probability'],
                market_probability=1 / bet['odds'],
                risk_level=risk_level,
                source=f"Enhanced_{betting_system['model_type'].upper()}",
                method_prediction=method_pred,
                bankroll=betting_system['bankroll']
            )
            
            if bet_id:
                saved_bets.append(bet_id)
                print(f"   {i}. ✅ {bet['fighter']} vs {bet['opponent']}")
            else:
                print(f"   {i}. ⚠️  Duplicate or error: {bet['fighter']}")
                
        elif bet['type'] == 'PARLAY':
            # Log parlay bet
            bet_id = tracker.log_bet_recommendation(
                event_name=EVENT_NAME,
                event_date=event_date,
                fighter=bet['fighters'],
                opponent="Multiple",
                bet_type=f"{bet['n_legs']}-LEG PARLAY",
                decimal_odds=bet['odds'],
                american_odds=int((bet['odds'] - 1) * 100) if bet['odds'] >= 2 else int(-100/(bet['odds']-1)),
                bet_size=bet['bet_size'],
                expected_value=bet['ev'],
                expected_return=bet['potential_profit'],
                model_probability=bet['probability'],
                market_probability=1 / bet['odds'],
                risk_level=risk_level,
                source=f"Enhanced_{betting_system['model_type'].upper()}",
                method_prediction=f"Parlay: {bet['n_legs']} legs",
                bankroll=betting_system['bankroll']
            )
            
            if bet_id:
                saved_bets.append(bet_id)
                print(f"   {i}. ✅ PARLAY: {bet['fighters']}")
            else:
                print(f"   {i}. ⚠️  Duplicate or error: Parlay")
    
    print(f"\n📊 Summary:")
    print(f"   • Recommendations: {len(betting_portfolio)}")
    print(f"   • New bets saved: {len(saved_bets)}")
    print(f"   • Duplicates skipped: {len(betting_portfolio) - len(saved_bets)}")
    
    return saved_bets

def check_duplicates_and_clean():
    """Check for and remove duplicate bets in the CSV."""
    print("\n🔍 CHECKING FOR DUPLICATES")
    print("=" * 50)
    
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Validate data integrity
    validation = tracker.validate_data()
    print(f"📊 Total records: {validation.get('total_records', 0)}")
    print(f"🔍 Data quality: {validation.get('data_quality', 'UNKNOWN')}")
    
    if validation.get('issues'):
        print(f"\n⚠️  Issues found:")
        for issue in validation['issues']:
            print(f"   • {issue}")
    
    # Remove duplicates
    duplicates_removed = tracker.remove_duplicate_recommendations()
    
    return duplicates_removed

def generate_quick_report():
    """Generate a quick performance report."""
    print("\n📈 PERFORMANCE ANALYSIS")
    print("=" * 50)
    
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Generate reports for different periods
    print("\n🔹 Last 7 days:")
    report_7d = tracker.generate_performance_report(days=7)
    
    print("\n🔹 Last 30 days:")
    report_30d = tracker.generate_performance_report(days=30)
    
    print("\n🔹 All time:")
    report_all = tracker.generate_performance_report(days=0)
    
    return report_all

def display_final_betting_card():
    """Display professional betting card."""
    print("\n" + "="*70)
    print("FINAL BETTING CARD")
    print("="*70)
    
    if not betting_portfolio:
        print("📭 No bets for this event")
        return
    
    print(f"Event: {EVENT_NAME}")
    print(f"Date: {datetime.now().strftime('%Y-%m-%d')}")
    print(f"Bankroll: ${betting_system['bankroll']:,.2f}")
    print(f"Model: {betting_system['model_type'].upper()}")
    print(f"Calibration: {'ON' if betting_system['calibration'] else 'OFF'}")
    
    # Separate singles and parlays
    singles = [b for b in betting_portfolio if b['type'] == 'SINGLE']
    parlays = [b for b in betting_portfolio if b['type'] == 'PARLAY']
    
    total_stake = sum(b['bet_size'] for b in betting_portfolio)
    total_potential = sum(b['potential_profit'] for b in betting_portfolio)
    
    print(f"\nTotal Stake: ${total_stake:.2f}")
    print(f"Potential Return: ${total_potential:.2f}")
    print(f"ROI: {(total_potential/total_stake)*100:.1f}%" if total_stake > 0 else "")
    
    # Display singles
    if singles:
        print(f"\n🎯 SINGLES ({len(singles)})")
        print(f"{'Fighter':<25} {'Odds':<8} {'Stake':<10} {'EV':<8} {'Profit':<10}")
        print("-" * 70)
        
        for bet in singles:
            # Determine marker based on odds/probability
            if bet.get('is_favorite', False):
                mark = "⭐"  # Favorite
            elif bet['probability'] < 0.3 and bet['ev'] > 0.10:
                mark = "🚨"  # Upset opportunity
            else:
                mark = "📊"  # Value bet
                
            print(f"{bet['fighter']:<25} {bet['odds']:<8.2f} "
                  f"${bet['bet_size']:<9.2f} {bet['ev']:<7.1%} "
                  f"${bet['potential_profit']:<9.2f} {mark}")
    
    # Display parlays
    if parlays:
        print(f"\n🎲 PARLAYS ({len(parlays)})")
        print(f"{'Fighters':<35} {'Odds':<8} {'Stake':<10} {'EV':<8} {'Profit':<10}")
        print("-" * 70)
        
        for bet in parlays:
            fighters_display = bet['fighters'][:35]  # Truncate if too long
            print(f"{fighters_display:<35} {bet['odds']:<8.2f} "
                  f"${bet['bet_size']:<9.2f} {bet['ev']:<7.1%} "
                  f"${bet['potential_profit']:<9.2f}")
    
    print("-" * 70)
    print(f"{'TOTAL':<35} ${total_stake:<9.2f} "
          f"{'':8} ${total_potential:<9.2f}")
    
    print("\n📋 LEGEND:")
    print("   ⭐ = Favorite | 🚨 = Upset opportunity | 📊 = Value bet")

def update_fight_results_interactive():
    """Interactive function to update fight results after the event."""
    print("\n🥊 UPDATE FIGHT RESULTS")
    print("=" * 50)
    
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Get pending bets for the current event
    pending = tracker.get_bets_by_event(EVENT_NAME)
    pending = pending[pending['actual_result'].isna()]
    
    if pending.empty:
        print("📭 No pending bets for this event")
        return
    
    print(f"Found {len(pending)} pending bets for {EVENT_NAME}")
    print("\nUpdate each result (WIN/LOSS/PUSH):")
    
    for _, bet in pending.iterrows():
        print(f"\n{bet['fighter']} vs {bet['opponent']}")
        print(f"   Stake: ${bet['bet_size']:.2f} @ {bet['odds_decimal']:.2f}")
        
        result = input("   Result (W/L/P or skip): ").upper()
        
        if result in ['W', 'WIN']:
            profit = bet['bet_size'] * (bet['odds_decimal'] - 1)
            tracker.update_fight_result(bet['bet_id'], 'WIN', profit)
        elif result in ['L', 'LOSS']:
            tracker.update_fight_result(bet['bet_id'], 'LOSS', -bet['bet_size'])
        elif result in ['P', 'PUSH']:
            tracker.update_fight_result(bet['bet_id'], 'PUSH', 0)
        else:
            print("   Skipped")

# ================== MAIN EXECUTION ==================
print("💾 Ready to save betting portfolio...\n")

# Check if betting_portfolio exists
if 'betting_portfolio' not in globals():
    print("❌ No betting portfolio found. Run Cell 4 first!")
else:
    # Display final card first
    display_final_betting_card()
    
    # Ask for confirmation
    if betting_portfolio:
        print("\n🎯 SAVE OPTIONS:")
        print("1. Save with duplicate detection (recommended)")
        print("2. Clean duplicates from existing records")
        print("3. Generate performance report")
        print("4. Skip saving")
        
        choice = input("\nSelect option (1-4): ")
        
        if choice == '1':
            saved_records = save_betting_portfolio_advanced()
            print("\n✅ BETTING ANALYSIS COMPLETE!")
            print("🎯 Good luck with your bets!")
            print("📝 Use update_fight_results_interactive() after the fights")
            
        elif choice == '2':
            duplicates_removed = check_duplicates_and_clean()
            print(f"\n✅ Cleaned {duplicates_removed} duplicate records")
            
        elif choice == '3':
            report = generate_quick_report()
            
        else:
            print("\n❌ Bets not saved")
    else:
        print("\n✅ Analysis complete - no bets recommended")

💾 Ready to save betting portfolio...


FINAL BETTING CARD
Event: UFC 319
Date: 2025-08-13
Bankroll: $17.00
Model: OPTIMIZED
Calibration: OFF

Total Stake: $0.53
Potential Return: $0.46
ROI: 86.9%

🎯 SINGLES (1)
Fighter                   Odds     Stake      EV       Profit    
----------------------------------------------------------------------
Michael Page              1.55     $0.49      7.5%    $0.27      ⭐

🎲 PARLAYS (1)
Fighters                            Odds     Stake      EV       Profit    
----------------------------------------------------------------------
Tim Elliott + Alex Hernandez + Gera 38.42    $0.04      476.4%  $0.19     
----------------------------------------------------------------------
TOTAL                               $0.53               $0.46     

📋 LEGEND:
   ⭐ = Favorite | 🚨 = Upset opportunity | 📊 = Value bet

🎯 SAVE OPTIONS:
1. Save with duplicate detection (recommended)
2. Clean duplicates from existing records
3. Generate performance report
4. Ski

In [ ]:
# 🏆 CELL 6: UPDATE FIGHT RESULTS (Standalone - Run After Event)
# ==============================================================
# This cell works independently - no need to run other cells first!

import pandas as pd
from datetime import datetime
import sys
sys.path.append('../..')

# Import the bet tracker
from ufc_predictor.betting.bet_tracking import BetTracker

def update_results_standalone():
    """Update fight results without running other cells."""
    print("🥊 UPDATE FIGHT RESULTS")
    print("=" * 50)
    
    # Initialize tracker
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Get all pending bets
    pending = tracker.get_pending_bets()
    
    if pending.empty:
        print("📭 No pending bets found")
        return
    
    print(f"Found {len(pending)} pending bet(s)\n")
    print("Enter result for each bet:")
    print("  • W or WIN = Your fighter won")
    print("  • L or LOSS = Your fighter lost")
    print("  • P or PUSH = No contest/draw")
    print("  • Press Enter to skip\n")
    
    updated_count = 0
    total_profit = 0
    
    for _, bet in pending.iterrows():
        print(f"\n🥊 {bet['fighter']} vs {bet['opponent']}")
        print(f"   Event: {bet['event']}")
        print(f"   Date: {bet['date_placed']}")
        print(f"   Stake: ${bet['bet_size']:.2f} @ {bet['odds_decimal']:.2f}")
        
        result = input("   Result (W/L/P or skip): ").strip().upper()
        
        if result in ['W', 'WIN']:
            profit = bet['bet_size'] * (bet['odds_decimal'] - 1)
            success = tracker.update_fight_result(bet['bet_id'], 'WIN', profit)
            if success:
                print(f"   ✅ WIN! Profit: +${profit:.2f}")
                updated_count += 1
                total_profit += profit
                
        elif result in ['L', 'LOSS', 'LOST']:
            loss = -bet['bet_size']
            success = tracker.update_fight_result(bet['bet_id'], 'LOSS', loss)
            if success:
                print(f"   ❌ LOSS. Lost: ${bet['bet_size']:.2f}")
                updated_count += 1
                total_profit += loss
                
        elif result in ['P', 'PUSH']:
            success = tracker.update_fight_result(bet['bet_id'], 'PUSH', 0)
            if success:
                print(f"   🤝 PUSH. Stake returned")
                updated_count += 1
        else:
            print("   ⏭️  Skipped")
    
    # Summary
    print("\n" + "=" * 50)
    print(f"📊 SESSION SUMMARY")
    print(f"   • Updated: {updated_count}/{len(pending)} bets")
    
    if updated_count > 0:
        print(f"   • Net P&L: ${total_profit:+.2f}")
        if total_profit > 0:
            print(f"   • Result: 🎉 PROFIT!")
        elif total_profit < 0:
            print(f"   • Result: 📉 Loss")
        else:
            print(f"   • Result: 🤝 Break even")
        
        print(f"\n✅ Results saved to betting_records.csv")
        
        # Quick performance check
        print("\n📈 RECENT PERFORMANCE (Last 30 days):")
        report = tracker.generate_performance_report(days=30)
        
    else:
        print("   • No bets were updated")

# Run the updater
update_results_standalone()

print("\n🎉 Thank you for using the Enhanced UFC Betting System!")
print("💪 Good luck with your next event!")